In [24]:
# import googlemaps
import requests
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import base64
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from dotenv import load_dotenv
import os

from io import BytesIO
from IPython.display import display
from PIL import Image
import io


In [3]:
# Suppress the warnings if you decide to disable SSL verification
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Initialize the client with your Google Maps API key
load_dotenv()
API_KEY = os.getenv('GOOGLE_API')
# gmaps = googlemaps.Client(key=API_KEY, requests_kwargs={'verify': False})

def merge_images(file1, file2, file3):
    """Merge three images into one, displayed side by side
    :param file1: path to first image file
    :param file2: path to second image file
    :param file3: path to third image file
    :return: the merged Image object
    """
    image1 = Image.open(file1)
    image2 = Image.open(file2)
    image3 = Image.open(file3)

    (width1, height1) = image1.size
    (width2, height2) = image2.size
    (width3, height3) = image3.size

    result_width = width1 + width2 + width3
    result_height = max(height1, height2, height3)

    result = Image.new('RGB', (result_width, result_height))
    result.paste(im=image1, box=(0, 0))
    result.paste(im=image2, box=(width1, 0))
    result.paste(im=image3, box=(width1 + width2, 0))
    return result

def get_route_df(start_address, end_address):
    # Get directions
    r = requests.get(f"https://maps.googleapis.com/maps/api/directions/json?key={API_KEY}&origin={start}&destination={end}&mode=driving", verify=False).json()
    
    df = pd.DataFrame(r['routes'][0]['legs'][0]['steps'])
    df['overview_polyline'] = r['routes'][0]['overview_polyline']['points']
    df['total_distance'] = r['routes'][0]['legs'][0]['distance']['text']
    df['total_duration'] = r['routes'][0]['legs'][0]['duration']['text']
    df['start_address'] = r['routes'][0]['legs'][0]['start_address']
    df['end_address'] = r['routes'][0]['legs'][0]['end_address']
    df['point_start_lat'] = df['start_location'].apply(lambda x: x['lat'])
    df['point_start_lng'] = df['start_location'].apply(lambda x: x['lng'])
    df['point_end_lat'] = df['end_location'].apply(lambda x: x['lat'])
    df['point_end_lng'] = df['end_location'].apply(lambda x: x['lng'])
    df['distance'] = df['distance'].apply(lambda x: x['text'])
    df['duration'] = df['duration'].apply(lambda x: x['text'])
    df['polyline'] = df['polyline'].apply(lambda x: x['points'])

    df.drop(columns=['end_location', 'start_location'], inplace=True)

    return df

def get_street_view(lat, lng, heading=0, fov=180, pitch=0):
    # Construct a URL for the Street View Static API

    street_view_api_url = f"https://maps.googleapis.com/maps/api/streetview?size=600x400&location={lat},{lng}&heading={heading}&fov={fov}&pitch={pitch}&key={API_KEY}"

    r = requests.get(street_view_api_url, verify=False)

    if r.status_code == 200:
        # Image was successfully retrieved
        file_path = f"pics/streetview_{lat}_{lng}.jpg"
        with open(file_path, 'wb') as file:
            file.write(r.content)
        return file_path
    else:
        print(f"No Street View available at lat: {lat}, lng: {lng}")
        return None

def main(start, end, name=None):
    route_df = get_route_df(start, end)

    for ix, row in route_df.iterrows():
        image_path = get_street_view(row['point_start_lat'], row['point_start_lng'])
        route_df.at[ix, 'image_path'] = image_path
    
    if name:
        route_df.to_csv(f'{name}_route.csv')
    return route_df


In [38]:
# Usage
start = "6801 S. Dale Mabry Hwy, MacDill AFB, FL"
end = "11076 Freedom Way, Seminole, FL"

# start = "Embassy of The United States of America, M44R+GJ6, Ahmet Krasniqi, Prishtina"
# end = "Prishtina International Airport “Adem Jashari”, H2FH+RRC, 14060"
#main(start, end, 'kosovo')

In [87]:
start = "6801 S. Dale Mabry Hwy, MacDill AFB, FL"
end = "11076 Freedom Way, Seminole, FL"

df = get_route_df(start, end)

for ix, row in df.iterrows():
    lat = row['point_start_lat']
    lng = row['point_start_lng']
    heading=0 
    fov=120
    street_view_api_url = f"https://maps.googleapis.com/maps/api/streetview/metadata?location={lat},{lng}&key={API_KEY}"
    r = requests.get(street_view_api_url, verify=False)
    if r.status_code == 200:
        df.at[ix, 'imagery_date'] = r.json()['date']
        for i in range(3):
            street_view_api_url = f"https://maps.googleapis.com/maps/api/streetview?size=600x400&location={lat},{lng}&heading={heading}&fov={fov}&pitch=0&key={API_KEY}&radius=50&return_error_code=true&source=outdoor"
            rr = requests.get(street_view_api_url, verify=False)
            file_path = f"pics/streetview_{lat}_{lng}_{heading}.jpg"
            
            with open(file_path, 'wb') as file:
                file.write(rr.content)
            df.at[ix, f'image_{i+1}'] = file_path
            heading += 120

    pano = merge_images(df['image_1'][ix], df['image_2'][ix], df['image_3'][ix])
    file_path = f"pics/streetview_{lat}_{lng}_pano.jpg"
    pano.save(f"pics/streetview_{lat}_{lng}_pano.jpg")
    df.at[ix, 'pano'] = file_path

In [88]:
df.to_csv('greg_route.csv')

In [ ]:
{'points': [{'curveNumber': 1, 'pointNumber': 1, 'pointIndex': 1, 'lon': -82.50713619999999, 'lat': 27.8682005, 
             'customdata': '1', 'bbox': {'x0': 585.8230859377649, 'x1': 587.8230859377649, 'y0': 293.3030885945426, 'y1': 295.3030885945426}}]}